In [14]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [15]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255
m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 13000
The number of features = 784


In [16]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [17]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [18]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [50] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [19]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 130


In [20]:
#Theta Initialization 
def theta_init(arch=50):
    theta = []
    arch = [arch]
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [21]:
def activation(x):
    return 1/(1+np.exp(-x))

In [22]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [23]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [24]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [25]:
epochs = []
train_accuracy = []
test_accuracy = []
train_time = []

In [ ]:
arch_test=[2]
for i in range(len(arch_test)):
    theta = theta_init(arch_test[i])
    fm = forward_prop(X_train, theta)
    
    epoch = 0
    start = time.time()
    cost_init = cost_total(X_train, theta, train_class_enc, m)

    while(True):
        count = 0

        print("Initial Cost for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            #delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
        epoch+=1 #Number of epochs
        cost_final = cost_total(X_train, theta, train_class_enc, m)
        print("Cost after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-07):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {}".format(epochs[-1]))
    print("The training time = {}sec".format(train_time[-1]))
    print("The training accuracy is = {}%".format(train_accuracy[-1]))
    print("The test accuracy is = {}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

Initial Cost for this epoch 0 = 3.6527652799792545
Error = 3.675390471504837
Error = 2.232230174756355
Error = 1.409912914369761
Cost after 1 epochs is = 1.3202483709279809
Initial Cost for this epoch 1 = 1.3202483709279809
Error = 1.3234153998791403
Error = 1.0016973983841402
Error = 0.8384964574610442
Cost after 2 epochs is = 0.8194834937964233
Initial Cost for this epoch 2 = 0.8194834937964233
Error = 0.8194028289702461
Error = 0.7341914846379775
Error = 0.6779502379353582
Cost after 3 epochs is = 0.6715063113220523
Initial Cost for this epoch 3 = 0.6715063113220523
Error = 0.6709417465416645
Error = 0.6358655269574106
Error = 0.608991323998137
Cost after 4 epochs is = 0.6062350082458561
Initial Cost for this epoch 4 = 0.6062350082458561
Error = 0.6055537333748191
Error = 0.5874538300908655
Error = 0.5720973677243762
Cost after 5 epochs is = 0.5707967053514637
Initial Cost for this epoch 5 = 0.5707967053514637
Error = 0.5700889850970571
Error = 0.5594654472893192
Error = 0.549659386

Error = 0.48164314633140887
Cost after 48 epochs is = 0.48166045812580865
Initial Cost for this epoch 48 = 0.48166045812580865
Error = 0.48096004644475626
Error = 0.4816403965418721
Error = 0.48157031933870686
Cost after 49 epochs is = 0.4815779229164408
Initial Cost for this epoch 49 = 0.4815779229164408
Error = 0.4808696208749466
Error = 0.4815572847000846
Error = 0.48150164686786595
Cost after 50 epochs is = 0.4814990501612809
Initial Cost for this epoch 50 = 0.4814990501612809
Error = 0.4807827703932722
Error = 0.4814773046625152
Error = 0.4814368102050996
Cost after 51 epochs is = 0.4814234861522672
Initial Cost for this epoch 51 = 0.4814234861522672
Error = 0.48069915122324997
Error = 0.4814000915088862
Error = 0.4813755197652803
Cost after 52 epochs is = 0.48135090588522206
Initial Cost for this epoch 52 = 0.48135090588522206
Error = 0.48061844567479556
Error = 0.48132530696126524
Error = 0.4813175120361228
Cost after 53 epochs is = 0.4812810102073612
Initial Cost for this epoch

Error = 0.4804507478414483
Cost after 94 epochs is = 0.479085500979644
Initial Cost for this epoch 94 = 0.479085500979644
Error = 0.4778059366238351
Error = 0.4780239434564522
Error = 0.48044968613812017
Cost after 95 epochs is = 0.4790407008083631
Initial Cost for this epoch 95 = 0.4790407008083631
Error = 0.47775724395932756
Error = 0.4779438316883589
Error = 0.4804479726907408
Cost after 96 epochs is = 0.4789964629101513
Initial Cost for this epoch 96 = 0.4789964629101513
Error = 0.4777100816773423
Error = 0.47786523687580557
Error = 0.48044549797102704
Cost after 97 epochs is = 0.47895276368531914
Initial Cost for this epoch 97 = 0.47895276368531914
Error = 0.47766439295079877
Error = 0.47778814686337356
Error = 0.4804421709021743
Cost after 98 epochs is = 0.4789095763500965
Initial Cost for this epoch 98 = 0.4789095763500965
Error = 0.4776201240698009
Error = 0.4777125408638982
Error = 0.4804379183130336
Cost after 99 epochs is = 0.47886687160145125
Initial Cost for this epoch 99 

Error = 0.47949679007187085
Cost after 140 epochs is = 0.47721940174197924
Initial Cost for this epoch 140 = 0.47721940174197924
Error = 0.4766439988452047
Error = 0.47536178251517236
Error = 0.4794614701688273
Cost after 141 epochs is = 0.4771777665098158
Initial Cost for this epoch 141 = 0.4771777665098158
Error = 0.47663462876078433
Error = 0.4753148595971047
Error = 0.47942597747574467
Cost after 142 epochs is = 0.4771360405245066
Initial Cost for this epoch 142 = 0.4771360405245066
Error = 0.47662561903966577
Error = 0.4752680329553826
Error = 0.47939034262900587
Cost after 143 epochs is = 0.4770942300695186
Initial Cost for this epoch 143 = 0.4770942300695186
Error = 0.4766169506164381
Error = 0.4752212919842793
Error = 0.47935459497426236
Cost after 144 epochs is = 0.4770523415779649
Initial Cost for this epoch 144 = 0.4770523415779649
Error = 0.47660860378690734
Error = 0.47517462685690576
Error = 0.4793187625205452
Cost after 145 epochs is = 0.4770103815845683
Initial Cost for

Error = 0.47792712349344596
Cost after 188 epochs is = 0.475218818788181
Initial Cost for this epoch 188 = 0.475218818788181
Error = 0.47627383865366923
Error = 0.4731418642094519
Error = 0.4779001227086751
Cost after 189 epochs is = 0.47517879504709926
Initial Cost for this epoch 189 = 0.47517879504709926
Error = 0.47626317298237675
Error = 0.4730963004704993
Error = 0.47787331153780727
Cost after 190 epochs is = 0.47513888255949904
Initial Cost for this epoch 190 = 0.47513888255949904
Error = 0.47625231837176274
Error = 0.4730508060801047
Error = 0.4778466802192148
Cost after 191 epochs is = 0.4750990824973068
Initial Cost for this epoch 191 = 0.4750990824973068
Error = 0.47624127388934223
Error = 0.4730053840899722
Error = 0.47782021886973414
Cost after 192 epochs is = 0.47505939596126245
Initial Cost for this epoch 192 = 0.47505939596126245
Error = 0.4762300383824396
Error = 0.4729600374728717
Error = 0.47779391752667494
Cost after 193 epochs is = 0.4750198239848092
Initial Cost fo

...

Error = 0.4491820748076773
Error = 0.4645134224873928
Cost after 9390 epochs is = 0.4564936145676072
Initial Cost for this epoch 9390 = 0.4564936145676072
Error = 0.45924790408515725
Error = 0.44917991712869
Error = 0.4645103647070386
Cost after 9391 epochs is = 0.45649297557738855
Initial Cost for this epoch 9391 = 0.45649297557738855
Error = 0.4592477337240054
Error = 0.44917775590426556
Error = 0.4645073001311467
Cost after 9392 epochs is = 0.4564923358284009
Initial Cost for this epoch 9392 = 0.4564923358284009
Error = 0.45924756365439073
Error = 0.4491755911244596
Error = 0.4645042287346389
Cost after 9393 epochs is = 0.456491695318272
Initial Cost for this epoch 9393 = 0.456491695318272
Error = 0.45924739387635144
Error = 0.44917342277930167
Error = 0.46450115049237656
Cost after 9394 epochs is = 0.45649105404462353
Initial Cost for this epoch 9394 = 0.45649105404462353
Error = 0.4592472243899128
Error = 0.44917125085879506
Error = 0.46449806537916105
Cost after 9395 epochs is = 

Cost after 9435 epochs is = 0.4564640741558215
Initial Cost for this epoch 9435 = 0.4564640741558215
Error = 0.4592405247901036
Error = 0.44907899676708796
Error = 0.4643653414727567
Cost after 9436 epochs is = 0.4564633985763164
Initial Cost for this epoch 9436 = 0.4564633985763164
Error = 0.4592403673682817
Error = 0.4490766653021758
Error = 0.4643619443242292
Cost after 9437 epochs is = 0.4564627221245792
Initial Cost for this epoch 9437 = 0.4564627221245792
Error = 0.459240210223641
Error = 0.44907432980675027
Error = 0.46435853917375497
Cost after 9438 epochs is = 0.4564620447979335
Initial Cost for this epoch 9438 = 0.4564620447979335
Error = 0.4592400533554053
Error = 0.44907199026962674
Error = 0.4643551259941907
Cost after 9439 epochs is = 0.4564613665936954
Initial Cost for this epoch 9439 = 0.4564613665936954
Error = 0.45923989676277416
Error = 0.4490696466795926
Error = 0.4643517047583691
Cost after 9440 epochs is = 0.456460687509175
Initial Cost for this epoch 9440 = 0.456

Error = 0.44897244748800286
Error = 0.46420793564346263
Cost after 9480 epochs is = 0.45643277059676934
Initial Cost for this epoch 9480 = 0.45643277059676934
Error = 0.45923370027412447
Error = 0.4489699277248008
Error = 0.46420416057831093
Cost after 9481 epochs is = 0.4564320530229559
Initial Cost for this epoch 9481 = 0.4564320530229559
Error = 0.45923355417308515
Error = 0.4489674034119767
Error = 0.4642003763106817
Cost after 9482 epochs is = 0.4564313344499365
Initial Cost for this epoch 9482 = 0.4564313344499365
Error = 0.4592334082863684
Error = 0.44896487453707123
Error = 0.46419658281360354
Cost after 9483 epochs is = 0.4564306148747411
Initial Cost for this epoch 9483 = 0.4564306148747411
Error = 0.45923326261182296
Error = 0.44896234108759475
Error = 0.464192780060143
Cost after 9484 epochs is = 0.45642989429439285
Initial Cost for this epoch 9484 = 0.45642989429439285
Error = 0.45923311714726034
Error = 0.44895980305102684
Error = 0.46418896802340553
Cost after 9485 epoch

Cost after 9525 epochs is = 0.4563994474763545
Initial Cost for this epoch 9525 = 0.4563994474763545
Error = 0.45922730138104656
Error = 0.448851635303445
Error = 0.464024357266852
Cost after 9526 epochs is = 0.4563986819087338
Initial Cost for this epoch 9526 = 0.4563986819087338
Error = 0.4592271622132519
Error = 0.448848892885832
Error = 0.46402013183979607
Cost after 9527 epochs is = 0.45639791520206374
Initial Cost for this epoch 9527 = 0.45639791520206374
Error = 0.45922702312123537
Error = 0.4488461453126034
Error = 0.4640158960363999
Cost after 9528 epochs is = 0.45639714735309495
Initial Cost for this epoch 9528 = 0.45639714735309495
Error = 0.4592268841008442
Error = 0.4488433925698037
Error = 0.4640116498333592
Cost after 9529 epochs is = 0.45639637835857266
Initial Cost for this epoch 9529 = 0.45639637835857266
Error = 0.4592267451478755
Error = 0.4488406346434428
Error = 0.4640073932074959
Cost after 9530 epochs is = 0.45639560821523495
Initial Cost for this epoch 9530 = 0

Error = 0.4638236311861427
Cost after 9571 epochs is = 0.4563630021335101
Initial Cost for this epoch 9571 = 0.4563630021335101
Error = 0.45922090639777907
Error = 0.4487199303359587
Error = 0.46381891785820484
Cost after 9572 epochs is = 0.45636218071421325
Initial Cost for this epoch 9572 = 0.45636218071421325
Error = 0.4592207656047353
Error = 0.44871693576785093
Error = 0.4638141932599246
Cost after 9573 epochs is = 0.45636135800008604
Initial Cost for this epoch 9573 = 0.45636135800008604
Error = 0.4592206246401975
Error = 0.4487139353643518
